In [1]:
##Imports and setup for Jupyter

import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from bs4 import BeautifulSoup
from IPython import display
import re
import plotly
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go




import nltk
from nltk import corpus
from nltk.collocations import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import wordnet
from nltk import punkt
from nltk.util import ngrams
from nltk import FreqDist

from datetime import datetime as dt
from datetime import date


from string import punctuation
from collections import Counter
import numpy as np
from numpy import random

import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

import json

%matplotlib inline


##function should allow to run in jupyter-- use show_app(#nameofapp) 

def show_app(app,  # type: dash.Dash
             port=9999,
             width=700,
             height=900,
             offline=True,
             style=True,
             **dash_flask_kwargs):
    """
    Run the application inside a Jupyter notebook and show an iframe with it
    :param app:
    :param port:
    :param width:
    :param height:
    :param offline:
    :return:
    """
    url = 'http://localhost:%d' % port
    iframe = '<iframe src="{url}" width={width} height={height}></iframe>'.format(url=url,
                                                                                  width=width,
                                                                                  height=height)
    display.display_html(iframe, raw=True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    if style:
        external_css = ["https://fonts.googleapis.com/css?family=Raleway:400,300,600",
                        "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
                        "http://getbootstrap.com/dist/css/bootstrap.min.css", ]

        for css in external_css:
            app.css.append_css({"external_url": css})

        external_js = ["https://code.jquery.com/jquery-3.2.1.min.js",
                       "https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js",
                       "http://getbootstrap.com/dist/js/bootstrap.min.js"]

        for js in external_js:
            app.scripts.append_script({"external_url": js})

    return app.run_server(debug=False,  # needs to be false in Jupyter
                          port=port,
                          **dash_flask_kwargs)



C:\Users\583185\AppData\Local\Continuum\Anaconda3\envs\py27\lib\site-packages\gensim\utils.py:860: UserWarning:

detected Windows; aliasing chunkize to chunkize_serial



## General Dash Notes

anything that starts with html. ... is generating pure HTML, anything that comes from the dcc. library is using things like Javascript, CSS and HTML

can customize the CSS styles of the app---app.css.append_css()

have to stop the kernal and restart each time you'd like to 'retest' the app--- guessing that it stays running so that you can make interactive changes to it.

style property is best to supply a dictionary to (colors example)

dcc.Graph renders interactive data viz using JavaScript graphing library.

can write text in markdown... set something text equal to a variable and then use dcc.Markdown function



## Call backs (interactivity) notes

need to import dash.depencies (input output)

describe inputs and outputs through a app.callback decorator

the inputs and outputs of the application are simply the properties of a particular component (so for example a dcc.Input)

don't set a value for the children property of the my-div component in the layout... when Dash starts it automatically calls all of the callbacks with the inital values of the input components in order to population the initial start

It's reactive programming



In [2]:
##Importing the final report

##can I call something from another jupyter notebook? Seems like we might need to either have a massive setup beforehand or
##save the other file as a .py script and then import like that

report=pd.read_csv('connect_report.csv')

report['Created Date']=pd.to_datetime(report['Created Date'])
report=report[pd.notnull(report['Created Date'])]
report=report[pd.notnull(report['Body'])]

#Removing HTML tags and codes
report['Body'] = report['Body'].apply(lambda x:BeautifulSoup(x))
report['Body'] = report['Body'].apply(lambda x:x.get_text())

#Removing tags/mentions
report['Body'] = report['Body'].apply(lambda x:re.sub('{@[\w\d]*}', '',x))

#Removing urls
report['Body'] = report['Body'].apply(lambda x:re.sub(r'^https?:\/\/.*[\r\n]*', '', x,flags=re.MULTILINE))

#Removing weird unicode characters
report['Body'] = report['Body'].apply(lambda x: x.encode('ascii', 'ignore'))


final_report=pd.DataFrame

#Use this function to create the final report
# Directions:
# 1. Set Group to a list of group names, a single name of a group, or All to use all groups
# 2. Set Type to a specific Feed Item Type or All
# 3. Set Start Date or leave blank(defaults to: )
# 4. Set End Date or leave blank(defaults to: )

# def report_generator(Group,Type,Date_start=pd.to_datetime("2017-01-01"),Date_finish=pd.to_datetime('today')):
#     if type(Group) == list:
#         final_report=report.loc[report['Name'].isin(Group)] 
#     elif (Group=='All'):
#         final_report=report
#     else:
#         final_report=report.loc[report['Name']==Group]
                                
#     if (Type=='All'):
#         final_report=final_report
#     else:
#         final_report=final_report.loc[final_report['Feed Item Type']==Type]
    
#     if(Date_start=="2017-01-01" and Date_finish== pd.to_datetime('today')):
#         final_report=final_report
#     else:
#         Date_start=pd.to_datetime(Date_start, utc=False)
#         Date_finish=pd.to_datetime(Date_finish, utc=False)
#         #alt approach-- not working but not sure why-- it worked it worked it worked! had to change the data type to all be annoying ass datatime64... down the line that might be a pain but for now it's dope
#         final_report=final_report[(final_report['Created Date'] > Date_start) & (final_report['Created Date']< Date_finish)]   
#     return(final_report)

# final_report=report_generator(['CPC+ All','NLT Internal Users'],'All',"2017-02-01","2017-03-01")


def report_generator(Group,Type):
    if type(Group) == list:
        final_report=report.loc[report['Name'].isin(Group)] 
    elif (Group=='All'):
        final_report=report
    else:
        final_report=report.loc[report['Name']==Group]
                                
    if (Type=='All'):
        final_report=final_report
    else:
        final_report=final_report.loc[final_report['Feed Item Type']==Type]     
    return(final_report)

final_report=report_generator(['CPC+ All','NLT Internal Users'],'All')

stop = stopwords.words('english')
stop.append("cpc+")
stop.append("hi")
stop.append("hello")
stop.append("also")
stop.append("anyone")
stop.append("et")
stop.append("please")

wnl = WordNetLemmatizer() 


def tokenizer(text):
 
    tokens_ = [word_tokenize(sent) for sent in sent_tokenize(text.lower())]
    
    tokens = []
    
    for token_by_sent in tokens_:
        tokens += token_by_sent
 
    tokens = list(filter(lambda t: t not in stop, tokens))
    tokens = list(filter(lambda t: t not in punctuation, tokens))
    tokens = list(filter(lambda t: t not in [u"'s", u"n't",u"'ve",u"'re",u"'d",u"'ll",u"'m", u"u", 
                                             u"...", u"''", u'``', u'\u2014', u'\u2026', u'\u2013'], tokens))
     
    filtered_tokens = []
    for token in tokens:
        token = wnl.lemmatize(token)
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
 
    filtered_tokens = list(map(lambda token: token.lower(), filtered_tokens))
    
    bigrm = nltk.bigrams(filtered_tokens)
    bi_tokens = []
    for a, b in bigrm:
        bi_tok = ' '.join((a, b))
        bi_tokens.append(bi_tok)
    
    filtered_tokens.extend(bi_tokens)
    
    return filtered_tokens

def single_day_tracker(report):
    #Create the top 10 list
    unlisted_report=report['tokens'].apply(lambda x: ', '.join(x))

    word_vectorizer_2=CountVectorizer(ngram_range=(1,1), analyzer='word')
    word_vectorizer_fit_2=word_vectorizer_2.fit_transform(unlisted_report)

    frequencies=sum(word_vectorizer_fit_2).toarray()[0]

    bigrams_counter=pd.DataFrame(frequencies,index=word_vectorizer_2.get_feature_names(),columns=['frequency'])
    f = bigrams_counter[bigrams_counter['frequency']>10]
    
   
    f=f.sort_values(by=['frequency'],ascending=False)
    top_words=f.head(15)
    
    top_words_listed=list(top_words.index)


    #Okay now figure out what the frequency was each day
    unique_dates=set(report['Created Date'])
    list_dates=sorted(list(unique_dates))
    list_df_name=list(xrange(0,len(list_dates)))
    list_df=[]
    new_df=pd.DataFrame
    for i in xrange(0,len(list_dates)):
        date_report=report.loc[report['Created Date']==list_dates[i]]
        unlisted_report_date=unlisted_report=date_report['tokens'].apply(lambda x: ', '.join(x))
        word_vectorizer_fit_date=word_vectorizer_2.fit_transform(unlisted_report_date)
        frequencies_date=sum(word_vectorizer_fit_date).toarray()[0]
        bigrams_counter_date=pd.DataFrame(frequencies_date,index=word_vectorizer_2.get_feature_names(),columns=['frequency'])
        bigrams_date_listed=list(bigrams_counter_date.index)
        finder=(bigrams_counter_date.index.isin(top_words.index))
        slimmed=(bigrams_counter_date[finder])
        slimmed['word']=slimmed.index
        slimmed['date']=list_dates[i]
        slimmed['date']=pd.to_datetime(slimmed['date'])
        list_df_name[i]=slimmed
        list_df.append(list_df_name[i])
    new_df=pd.concat(list_df)
    new_df.index=xrange(0,len(new_df['word']))
    return(new_df)

##MOVED INTO REACTIVE SECTION BELOW
            
# top_words=single_day_tracker(final_report)


    
# #Bigrams/collocation scoring
# bigram_measures=nltk.collocations.BigramAssocMeasures()
# finder_2=BigramCollocationFinder.from_documents(final_report['tokens'])
# score_collocation_2=finder_2.nbest(bigram_measures.pmi,5)

# scored = finder_2.score_ngrams(bigram_measures.pmi)
# sorted_scored=sorted(bigram for bigram, score in scored)



#Single word finder
def single_word_finder(x,y,mode='single'):
    scores_e=list(enumerate(scored))
    index_list=[]
    final_list=[]
    if(mode=='double'):
        for i, v in enumerate(scored):
            if v[0] == (x,y):
                return scores_e[i]
            elif v[0] == (y,x):
                return scores_e[i]
    elif(mode=='single'):
        for i, v in enumerate(scored):
            if(scores_e[i][1][0][0]) == x or scores_e[i][1][0][1] == x or scores_e[i][1][0][0]==y or scores_e[i][1][0][1]==y:
                index_list.append(i)
        for j in index_list:
            final_list.append(scores_e[j])
        return final_list

def graphing_single_word_finder(word1,word2='',mode='single'):
    graphing_data=single_word_finder(word1,'','single')
    graphing_data=pd.Series(graphing_data)
    score_list=[]
    non_word1_list=[]
    for i,v in graphing_data.iteritems():
        if(graphing_data[i][1][0][0]!=word1):
            non_word1_list.append(graphing_data[i][1][0][0])
        elif(graphing_data[i][1][0][1])!=word1:
            non_word1_list.append(graphing_data[i][1][0][1])
        score_list.append(graphing_data[i][1][1])
    final_graphing=pd.concat([pd.Series(non_word1_list),pd.Series(score_list)],axis=1)
    final_graphing.columns=['Word','PMI_Score']
    sns.mpl.rc("figure", figsize=(9,4))
    word_graph=sns.stripplot(x="Word", y="PMI_Score", data=final_graphing, size = 8)
    loc, labels = plt.xticks()
    word_graph.set_xticklabels(labels, rotation=90)
    return(final_graphing)

def graphing_labels(word1,word2='',mode='single'):
    graphing_data=single_word_finder(word1,'','single')
    graphing_data=pd.Series(graphing_data)
    score_list=[]
    non_word1_list=[]
    for i,v in graphing_data.iteritems():
        if(graphing_data[i][1][0][0]!=word1):
            non_word1_list.append(graphing_data[i][1][0][0])
        elif(graphing_data[i][1][0][1])!=word1:
            non_word1_list.append(graphing_data[i][1][0][1])
        score_list.append(graphing_data[i][1][1])
    final_graphing=pd.concat([pd.Series(non_word1_list),pd.Series(score_list)],axis=1)
    final_graphing.columns=['Word','PMI_Score']
    return(final_graphing)

C:\Users\583185\AppData\Local\Continuum\Anaconda3\envs\py27\lib\site-packages\bs4\__init__.py:181: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\Users\583185\AppData\Local\Continuum\Anaconda3\envs\py27\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")




In [ ]:
##Setting up table generator example
def generate_table(dataframe, maxrows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), maxrows))]
    )


In [17]:
##Text Example

example_text='''
## This is where we would write an intro.

Maybe something about how to use the dashboard, or how to get it to run.
Explanation of scores, things like that.
'''

In [ ]:
##Setting up a test environment



app=dash.Dash()



colors={
    'background': '#111111',
    'text': '#7FDBFF'
}

app.layout=html.Div(children=
    [html.H1(children='This is the title pane',
             style={
                 'textAlign': 'center', 'color':colors['text']}   ),
    dcc.Markdown(children=example_text), 
    html.H2(children='Please use these inputs to customize your report',
            style={
                 'textAlign': 'center', 'color':colors['text']} ),
    dcc.DatePickerRange(
        id='date_selector',
        min_date_allowed=dt(2017,1,1),
        end_date=dt.today()),
    html.Table(id='table'), 
#Need to work on spacing here and then figure out how to use them as arguments for all of the functions  
#     html.Label('First word finder'),
#     dcc.Input(
#         id='first_word_finder',
#         value='',
#         type='text'),
#     html.Label('Second word finder'),
#     dcc.Input(value='If you would like to find the intersection of the first word and another please enter here',type='text'),
#     ##Try to build the graph here 
#     dcc.Graph(
#         id='test_graph', 
        
#     ),
     
    
     
     html.Div(id='output-container-date-selector',style={'display': 'none'}),
     html.Div(id='output-container-top-words',style={'display':'none'}),
     html.Div(id="output-container-scored",style={'display':'none'})
     
    
])



#Need to move things that are setup/dependent on reactive inputs here into here
@app.callback(
    dash.dependencies.Output('output-container-date-selector','children'),
[dash.dependencies.Input('date_selector','start_date'),
 dash.dependencies.Input('date_selector','end_date')]
)
##going to need to do this in a bunch of different functions?
def update_report(final_report,input1,input2):
    if(input1=="2017-01-01" and input2== pd.to_datetime('today')):
        final_report_reactive=final_report
    else:
        start=pd.to_datetime(input1, utc=False)
        end=pd.to_datetime(input2, utc=False)
        final_report_reactive=final_report[(final_report['Created Date'] > start) & (final_report['Created Date']< end)]   
    return(final_report_reactive.to_json())

#Testing on the table
@app.callback(dash.dependencies.Output('table','children'),
             [dash.dependencies.Input('output-container-date-selector','children')]
             )
def update_table(input1):
    final_report_updated=pd.read_json(input1)
    reactive_table=create_table(final_report_updated)
    return(reactive_table)

# @app.callback(dash.dependencies.Output('output-container-top-words','children'),
#              dash.dependencies.Input('output-container-date-selector','children')
#              )
# def update_top_words(input):
#     top_words_reactive=single_day_tracker(input)
#     return(top_words_reactive.to_json())

# @app.callback(dash.dependencies.Output('output-container-scored','children'),
#             [dash.dependencies.Input('output-container-top-words','children'),
#               dash.dependencies.Input('output-container-date-selector','children')]
#              )
# def update_scores(input1): 
# #Bigrams/collocation scoring
#     bigram_measures=nltk.collocations.BigramAssocMeasures()
#     finder_2=BigramCollocationFinder.from_documents(input2['tokens'])
#     score_collocation_2=finder_2.nbest(bigram_measures.pmi,5)

#     scored = finder_2.score_ngrams(bigram_measures.pmi)
#     sorted_scored_reactive=sorted(bigram for bigram, score in scored)
#     return(sorted_scored_reactive)


# @app.callback(
#     dash.dependencies.Output('test_graph','figure'),
#     [dash.dependencies.Input('first_word_finder','value'),
#     dash.dependencies.Input('output-container-scored')]
# )

# def update_graph(input1,input2):

#     pmi_score=graphing_single_word_finder(input1,'',mode='single')
#     g_labels=graphing_labels(input2,'',mode='single')
#     g_labels=g_labels['Word'].tolist()
#     return{
#             'data': [
#                 go.Scatter(
#                 x=pmi_score['Word'],
#                 y=pmi_score['PMI_Score'],
#                 mode='markers',
#                     marker=dict(
#                     color=range(0,len(pmi_score)),
#                     cmin=0,
#                     cmax=28,
#                     colorscale='Jet'
#                 )
#                 )
#                 ],
#             'layout':go.Layout(
#                 xaxis={'title':'PMI for Searched words','ticktext':g_labels}
#                 )
#    }


show_app(app)

In [31]:
##Run the app in broswer instead

app=dash.Dash()

report=pd.read_csv('connect_report.csv')

report['Created Date']=pd.to_datetime(report['Created Date'])
report=report[pd.notnull(report['Created Date'])]
report=report[pd.notnull(report['Body'])]

#Removing HTML tags and codes
report['Body'] = report['Body'].apply(lambda x:BeautifulSoup(x))
report['Body'] = report['Body'].apply(lambda x:x.get_text())

#Removing tags/mentions
report['Body'] = report['Body'].apply(lambda x:re.sub('{@[\w\d]*}', '',x))

#Removing urls
report['Body'] = report['Body'].apply(lambda x:re.sub(r'^https?:\/\/.*[\r\n]*', '', x,flags=re.MULTILINE))

#Removing weird unicode characters
report['Body'] = report['Body'].apply(lambda x: x.encode('ascii', 'ignore'))

final_report=pd.DataFrame(data=report)






colors={
    'background': '#111111',
    'text': '#7FDBFF'
}

app.layout=html.Div(children=
    [html.H1(children='This is the title pane',
             style={
                 'textAlign': 'center', 'color':colors['text']}   ),
    dcc.Markdown(children=example_text), 
    html.H2(children='Please use these inputs to customize your report',
            style={
                 'textAlign': 'center', 'color':colors['text']} ),
    dcc.DatePickerRange(
        id='date_selector',
        min_date_allowed=dt(2017,1,1),
        start_date=dt(2017,1,1),
        end_date=dt.today()),
    html.Table(id='table'), 
#Need to work on spacing here and then figure out how to use them as arguments for all of the functions  
#     html.Label('First word finder'),
#     dcc.Input(
#         id='first_word_finder',
#         value='',
#         type='text'),
#     html.Label('Second word finder'),
#     dcc.Input(value='If you would like to find the intersection of the first word and another please enter here',type='text'),
#     ##Try to build the graph here 
#     dcc.Graph(
#         id='test_graph', 
        
#     ),
     
    
     
     html.Div(id='output-container-date-selector',style={'display': 'none'}),
     html.Div(id='output-container-top-words',style={'display':'none'}),
     html.Div(id="output-container-scored",style={'display':'none'})
     
    
])



#Need to move things that are setup/dependent on reactive inputs here into here
@app.callback(
    dash.dependencies.Output('output-container-date-selector','children'),
[dash.dependencies.Input('date_selector','start_date'),
 dash.dependencies.Input('date_selector','end_date')]
)
##going to need to do this in a bunch of different functions?
def update_report(input1,input2):
    start=pd.to_datetime(input1,utc=False)
    end=pd.to_datetime(input2,utc=False)
    if(start=="2017-01-01" and end== pd.to_datetime('today')):
        final_report_reactive=final_report
    else:
        final_report_reactive=final_report[(final_report['Created Date'] > start) & (final_report['Created Date']< end)]   
        json_save=final_report_reactive.to_json(orient='index')
        json_string=(json.dumps(json_save))
        return(json_string)

#Testing on the table
@app.callback(dash.dependencies.Output('table','children'),
             [dash.dependencies.Input('output-container-date-selector','children')]
             )
def update_table(input1):
    final_report_updated=pd.read_json(json.loads(input1),orient='index')
    reactive_table=html.Table(final_report_updated)
    return(reactive_table)





if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Dec/2017 10:10:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2017 10:10:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2017 10:10:53] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2017 10:10:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2017 10:10:55] "GET /favicon.ico HTTP/1.1" 200 -
[2017-12-07 10:10:56,141] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\583185\AppData\Local\Continuum\Anaconda3\envs\py27\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\583185\AppData\Local\Continuum\Anaconda3\envs\py27\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\583185\AppData\Local\Continuum\Anaconda3\envs\py27\lib\site-packages\flask\app.py", line 1517, i

127.0.0.1 - - [07/Dec/2017 10:10:56] "POST /_dash-update-component HTTP/1.1" 500 -


In [23]:
final_report_test=final_report[(final_report['Created Date']>"2017-04-01") & (final_report['Created Date'] < "2017-05-01")]
json_test=final_report_test.to_json()

json_string=(json.dumps(json_test))

pd.read_json(json.loads(json_string))

pd.to_datetime("1/1/2017")

Timestamp('2017-01-01 00:00:00')